In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection

In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)
test['Fare'] = test['Fare'].fillna(train['Fare'].median())
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)
predictors = ['Pclass', 'IsFemale', 'Age', 'Fare']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values
submission = pd.DataFrame(test.iloc[:, 0])

In [7]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [9]:
gnb = GaussianNB()
nb_pred = gnb.fit(X_train, y_train).predict(X_test)
nb_pred = pd.DataFrame(nb_pred)
nb_pred.columns = ['Survived']
nb_submission = pd.concat([submission, nb_pred], axis=1)
nb_submission.to_csv('nb_submission.csv',index=False)

In [12]:
rfModel = RandomForestClassifier()
rfModel.fit(X_train, y_train)
rf_pred = rfModel.predict(X_test)
rf_pred = pd.DataFrame(rf_pred)
rf_pred.columns = ['Survived']
rf_submission = pd.concat([submission, rf_pred], axis=1)
rf_submission.to_csv('rf_submission.csv',index=False)

In [16]:
model = LogisticRegression()
model.fit(X_train, y_train)

kfold = model_selection.KFold(n_splits=10, random_state=7)
scoring = 'accuracy'
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
print("Accuracy: " , results.mean()) 

y_predict = model.predict(X_test)
y_predict = pd.DataFrame(y_predict)
y_predict.columns = ['Survived']

submission = pd.concat([submission, y_predict], axis = 1)
submission.to_csv('lr_submission.csv',index=False)

Accuracy:  0.786754057428
